# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.01,95,100,18.87,TF,1727745335
1,1,tegueste,28.5167,-16.3167,19.81,87,75,7.72,ES,1727745336
2,2,princeville,45.7668,-61.2985,14.06,89,98,1.34,CA,1727745337
3,3,puerto natales,-51.7236,-72.4875,2.90,88,100,5.17,CL,1727745338
4,4,gaspe,48.8334,-64.4819,8.29,73,1,2.45,CA,1727745339


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',        # Longitude
    'Lat',        # Latitude
    size='Humidity',  # Size of the points based on humidity
    color='City',
    geo=True,     # Enable geographic plotting
    tiles='OSM',  # Use OpenStreetMap tiles
    frame_width=800,
    frame_height=600,
    title='City Humidity Map',
    legend='right'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,dhanera,24.5167,72.0167,26.66,85,0,2.16,IN,1727745345
20,20,al bawiti,28.3492,28.8659,21.62,60,0,4.03,EG,1727745358
42,42,pascagoula,30.3658,-88.5561,26.00,78,0,0.00,US,1727745383
45,45,sao vicente,-23.9631,-46.3919,22.75,26,0,0.89,BR,1727745387
158,158,ha'il,27.5219,41.6907,23.01,27,0,1.03,SA,1727745517


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,dhanera,IN,24.5167,72.0167,85,
20,al bawiti,EG,28.3492,28.8659,60,
42,pascagoula,US,30.3658,-88.5561,78,
45,sao vicente,BR,-23.9631,-46.3919,26,
158,ha'il,SA,27.5219,41.6907,27,
177,cambria,US,35.5641,-121.0808,54,
201,maintirano,MG,-18.0667,44.0167,87,
232,bayji,IQ,34.9311,43.4870,18,
259,manfalut,EG,27.3104,30.9700,57,
280,khost,AF,33.3395,69.9204,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']  # Assuming coordinates are in the format (latitude, longitude)
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2.5/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dhanera - nearest hotel: No hotel found
al bawiti - nearest hotel: No hotel found
pascagoula - nearest hotel: No hotel found
sao vicente - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found
cambria - nearest hotel: No hotel found
maintirano - nearest hotel: No hotel found
bayji - nearest hotel: No hotel found
manfalut - nearest hotel: No hotel found
khost - nearest hotel: No hotel found
hun - nearest hotel: No hotel found
mulanje - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
yamada - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
show low - nearest hotel: No hotel found
turbat - nearest hotel: No hotel found
jatai - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
8,dhanera,IN,24.5167,72.0167,85,No hotel found
20,al bawiti,EG,28.3492,28.8659,60,No hotel found
42,pascagoula,US,30.3658,-88.5561,78,No hotel found
45,sao vicente,BR,-23.9631,-46.3919,26,No hotel found
158,ha'il,SA,27.5219,41.6907,27,No hotel found
177,cambria,US,35.5641,-121.0808,54,No hotel found
201,maintirano,MG,-18.0667,44.0167,87,No hotel found
232,bayji,IQ,34.9311,43.4870,18,No hotel found
259,manfalut,EG,27.3104,30.9700,57,No hotel found
280,khost,AF,33.3395,69.9204,33,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",          # Longitude column
    "Lat",           # Latitude column
    geo=True,             # Enable geographic plotting
    tiles="EsriImagery",  # Choose the tile layer
    size="humidity",      # Size of the points based on humidity
    scale=0.1,            # Adjust the scale for visibility
    color="City",         # Color points based on city
    frame_width=800,      # Set the width of the frame
    frame_height=600,      # Set the height of the frame
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)